# Data Preprocessing steps to make it fit for applying machine learning algorithms.

### 1. Importing Libraries for Data Preprocessing

In [86]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# 'Numpy' is used for mathematical operations on large, multi-dimensional arrays and matrices
# 'Pandas' is used for data manipulation and analysis
# 'Matplotlib' is a data visualization library for 2D and 3D plots, built on numpy
# 'Seaborn' is based on matplotlib; used for plotting statistical graphics

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [87]:
df = pd.read_csv("../input/ml-dataset/lending-club-loans.csv", low_memory=False) #Dataset

In [88]:
# Checking the Dimensions of our dataset

df.shape

In [89]:
df.columns

To get familiarize ourselves with this dataset, and understand what these columns represent.

In [90]:
description = pd.read_csv('../input/lcdata/LCDataDictionary.csv').dropna()
description.style.set_properties(subset=['Description'], **{'width' :'850px'})

In [91]:
df.info()

In [92]:
fig = plt.figure(figsize=(15,10))
sns.heatmap(df.isna(),cmap='inferno')

It is evident from the above heatmap that our dataset contains a lot of missing values and we can not use feature that has so many missing values.

More Darker Color represent the data is filled.

Another thing I want to examine is that how many loans have a default loan status in comparison to other loans.

So, 
A common thing to predict in datasets like these are if a new loan will get default or not. 
I will be keeping loans with default status as my target variable.

In [93]:
df['loan_status'].value_counts()

In [94]:
target = [1 if i=='Default' else 0 for i in df['loan_status']]
df['target'] = target
df['target'].value_counts()

In [95]:
nulls = pd.DataFrame(round(df.isnull().sum()/len(df.index)*100,2),columns=['null_percent'])
sns.barplot(x='index',y='null_percent',data=nulls.reset_index())
nulls[nulls['null_percent']!=0.00].sort_values('null_percent',ascending=False)

### There are several columns that fit into one of the following categories:

Unneccesary data : URL to view the loan. <br>
Redundant data : Loan description may be useful to some, but loan purpose fits our needs <br>
User provided information : Employer titles may offer some insight into employment industry.<br>
Operational data : The next payment date for the loan at the time the data was generated is not relevant to us 

In [96]:
# Drop unneccesary columns
df = df.drop(['url', 'desc', 'policy_code', 'last_pymnt_d', 'next_pymnt_d', 'earliest_cr_line', 'emp_title'], axis=1)
df = df.drop(['id', 'title', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'zip_code'], axis=1)


In [97]:
df['member_id'].value_counts().head(5)

Since customer has taken loan again, we cannot drop member id.

Some rows that we decided were not relevant to our needs. <br> 
The 'loan_status' column is the source of answer to the main question that if people are paying the loans they take out. 

Some records with a loan_status of  "Does not meet the credit policy". These may be older loans that would simply not be accepted under LendingClubs current criteria. 
As these data points will provide no value moving forward, I have excluded them from our data. 

Similiarily, recently issued loans could mislead our analysis, as no payment has been expected yet.

In [98]:
i = len(df)
df = pd.DataFrame(df[df['loan_status'] != "Does not meet the credit policy. Status:Fully Paid"])
df = pd.DataFrame(df[df['loan_status'] != "Does not meet the credit policy. Status:Charged Off"])
df = pd.DataFrame(df[df['loan_status'] != "Issued"])
df = pd.DataFrame(df[df['loan_status'] != "In Grace Period"])
a = len(df)
print(f"I Dropped {i-a} rows, a {((i-a)/((a+i)/2))*100}% reduction in rows")

### Data Type Handling

In [99]:
# formating int_rate and term to proper datatypes
df["int_rate"] =df["int_rate"].str.rstrip('%').astype('float')
df["term"]=df["term"].str.rstrip(' months').astype('float')


### Exploring the Data

Looking the Distribution of Data types

In [100]:
df.dtypes

In [101]:
df.info()

In [102]:
# Let us see how many Object type features are actually Categorical
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

#### The distribution of Employement Lengths.

In [103]:
sns.set(rc={'figure.figsize':(15,8)})
sns.countplot(df['emp_length'],palette='inferno')
plt.xlabel("Length")
plt.ylabel("Count")
plt.title("Distribution of Employement Length For Issued Loans")
plt.show()

#### It can be seen that people who have worked for 10 or more years are more likely to take loans

In [104]:
sns.set(rc={'figure.figsize':(15,6)})
sns.boxplot(x='loan_amnt', y='loan_status', data=df)

# Cleaning The Data

In [105]:
df.columns

There are some columns/features that are not required. I have already dropped those features.

In [106]:
df.shape

In [107]:
df.head(5)

#### Drop columns that have more than 75% null values

In [108]:
nulls = pd.DataFrame(round(df.isnull().sum()/len(df.index)*100,2),columns=['null_percent'])
drop_cols = nulls[nulls['null_percent']>75.0].index
df.drop(drop_cols, axis=1, inplace=True)

In [109]:
df.shape

In [110]:
df.head(5)

In [111]:
df.columns

Now convert date object columns to integer years or months so that we can easily encode other categorical features without exhausting our resources. 

For filling the dates, I using the most used dates in that feature

In [112]:
df.drop(['loan_status'],axis=1,inplace=True)

In [113]:
categorical = []
for column in df:
    if df[column].dtype == 'object':
        categorical.append(column)
categorical

In [114]:
df.shape

In [115]:
df['mths_since_last_delinq'].fillna(df['mths_since_last_delinq'].median(), inplace=True)

In [116]:
def remove_invalid_entries(data):
    # these indexes contains id as invalid type and having rest of the features null
    idxs= data[data['member_id'].isnull()].index.tolist()
    data= data.drop(idxs, axis=0)
    return data

def remove_constant_featurues(data):
    cols= [col for col in df.columns if len(df[col].unique())==1]
    data = data.drop(cols, axis=1)
    return data


def handle_outliers(data, column):
    '''
    criteria for outliers
    count for outliers
    if count<1%
        remove entries
        
    else if 1%< count <15% 
        subtitute with medians
    
    else 
        log features
    ''' 
    Q1=df[column].quantile(0.25)
    Q3=df[column].quantile(0.75)
    IQR=Q3-Q1
    idxs= data[((data[column]<(Q1-1.5*IQR)) | (data[column]>(Q3+1.5*IQR)))].index.tolist()
    
    if(len(data[col].unique()) < 4 or len(idxs)==0):
        return data
    
    if(len(idxs)<(len(data)*0.01)):
        print(f"Removing {len(idxs)} entries in {col}")
        data= data.drop(idxs, axis=0)
    
    elif ((len(data)*0.01)< len(idxs) and len(idxs) <= (len(data)*0.10)):
        print(f"Substitued with Mean in {col}")
        data[column][idxs]= data[column].mean()
        
    elif len(idxs) > (len(data)*0.10):
        sns.displot(data, x= col)
        print(f"Log transformation is done in {col}")
        data[column] = np.log(data[column])
        sns.displot(data, x= col)
    
    return data


In [117]:
df = df.loc[:,~df.T.duplicated(keep='first')]  # removing duplicate columns

df= remove_constant_featurues(df) # removing constant features
df.drop_duplicates(keep=False, inplace=True) # removing duplicate rows 

In [118]:
mapping_dict= {"emp_length": 
    {"10+ years": 10,"9 years": 9,"8 years": 8,"7 years": 7,"6 years": 
        6, "5 years": 5,"4 years": 4,"3 years": 3,"2 years": 2,"1 year": 1,"< 1 year": 0,"n/a": 0
    },
 
 "grade":{"A": 1,"B": 2,"C": 3,"D": 4,"E": 5,"F": 6,"G": 7},
 
 "loan_status": {
        "Fully Paid": 1, "Charged Off": 0
     }
}

In [119]:
df["int_rate"]

In [120]:
df.info()

In [121]:
num_features = [col for col in df.columns if df[col].dtypes != "object"]
cat_features = [col for col in df.columns if df[col].dtypes == "object"]

In [122]:
# mapping considerate categorical features to their corresponding numerical values
df = df.replace(mapping_dict)

for col in num_features:
        
        df = handle_outliers(df, col)
    

for col in num_features:
    df[col].fillna(df[col].mean(), inplace= True) # filling null with mean in numerical_features
    
for col in cat_features:
    df[col].fillna(df[col].mode(), inplace= True)

In [124]:
for col in cat_features:
    if df[col].dtypes=="object":
        df[col] = df[col].astype("category")
        df[col] = df[col].cat.codes